In [11]:
import os

import pandas as pd

import numpy as np 
from pathlib import Path
from collections import Counter

from tqdm import tqdm
import json
import pandas as pd

import re

import matplotlib.pyplot as plt
import seaborn as sns

import spacy
nlp = spacy.load('en_core_web_sm')

from spacytextblob.spacytextblob import SpacyTextBlob
nlp.add_pipe('spacytextblob')

from string import punctuation as PUNCTUATION
from spacy.lang.en.stop_words import STOP_WORDS

from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix

In [12]:
# Changing Directory
os.chdir('/Users/annettedblackburn/Desktop/Data_Analytics_Bootcamp/Module 20 - Final (Group) Project')

In [13]:
os.listdir(os.curdir)

['.DS_Store',
 'crude-oil-price.csv',
 'PET_PRI_GND_DCUS_NUS_W.csv',
 'archive(2).zip',
 'archive(1).zip',
 'twitter_NLP.ipynb',
 'gas_oil_ML.ipynb',
 'tweets_clean.csv',
 '.ipynb_checkpoints']

In [14]:
twitter_df = pd.read_csv("tweets_clean.csv")
twitter_df.head()

,Unnamed: 0,2022-11-24,2022-11-25,2022-11-26,2022-11-27,2022-11-28,2022-11-29,2022-11-30,2022-12-01,2022-12-02,2022-12-03,2022-12-04,2022-12-05
0,0,@BellaWallerstei They have been paying oil and...,@TrainerGonzo @Belinda71555585 It's that easy ...,@HouseGOP They tried to stick the gas price hi...,@GilHough @KamalaHarris https://t.co/LBgWDBW9xs,"@ChrisMartzWX For well over 40 years, I have p...",@POTUS Except that gas prices were on the rise...,@FinancialReview simply nonsense. Don't let th...,@BBCNews You have a responsibility to explain ...,@GustavoBolivar Es una afrenta para la industr...,@SenateGOP Except the oil and gas companies do...,@BladeoftheS He thinks nurses shouldn’t get a ...,Just When We Thought King Coal Was a Pauper\n#...
1,1,@BladeoftheS 3 The major difference between UK...,@davidmweissman Clueless! Right he inherited e...,@AnnieForTruth Yep. I remember when an attenda...,@NikkiHaley I agree use our own oil and coal a...,@WhiteHouse His executive orders on the energy...,@robreiner You really need help. Why are you s...,"@GavinNewsom You could end the gas tax, but yo...",@Marmee33 https://t.co/MeXmoIOgc9,@JDCocchiarella Jack; I can answer that questi...,@GOP And Biden has created more jobs in 2 year...,@POTUS No thanks on your nuclear energy deal d...,Alberta officials withholding list of 100s of ...
2,2,¡Oil &amp; Gas Reporter está disponible! https...,@laurenboebert Magic money? You mean from the ...,"@RBReich -\n...and the NATO Supremacists said,...",Nothing is being offered near Utah’s oil and g...,@manuel_rdgn @SEGOB_mx @adan_augusto @calderon...,TCS is a proud sponsor of the Oil &amp; Gatsby...,@POTUS Gas prices are down only because you ar...,@Sadiksmart7 @mohamedamiin2 @Qatari You can't ...,@diegowainstein El colectivo trucho de feminis...,"@SandyHasCandy @TomTSEC And somehow, in your b...","@Julesmerch @ABC Has nothing to do with oil, n...",@GavinNewsom Californians are the only state p...
3,3,@binnahar85 Europe wants to boycott Qatar? Are...,@thetruthsucks12 @haliwell51 @gmbutts No maybe...,@Princemoye1 I thought its about those that ki...,@Scott19490 @isaiahrmartin Yeah because relian...,rtr: U.S. seeks to limit flaring and methane l...,Should California tax oil profits? Gas spike h...,https://t.co/HEMuIIrFin Oil “Quality News Link...,¡Oil &amp; Gas Reporter está disponible! https...,"Cut off from Russian gas and oil imports, much...",@MichelleSteelCA If only our idiot Prime Minis...,¡Oil &amp; Gas Reporter está disponible! https...,Two energy industry groups filed a lawsuit tod...
4,4,@TheoFleury14 This was always about $$ and the...,You are full of it. You shut down our clean cr...,The big story. Proof that Westminster has been...,@RepBoebert Lauren Boebert’s income is almost ...,¡Oil &amp; Gas Reporter está disponible! https...,This is a classic example of Washington corrup...,@SenatorLankford You do know that US is the la...,#Gold #Silver and #Crypto update for 12/01/22 ...,¡Oil &amp; Gas Reporter está disponible! https...,PUMP AMERICAN OIL/NG NOW TO SAVE THE US ECONOM...,"Not to be that guy, but, this fossil fuel geop...",@GavinNewsom It's time for California to stop ...


In [15]:
twitter_df.tail()
# current twitter_df format: 3310, 13 (rows, columns)
# new twitter_df format: 43043, 1 (rows, columns)

,Unnamed: 0,2022-11-24,2022-11-25,2022-11-26,2022-11-27,2022-11-28,2022-11-29,2022-11-30,2022-12-01,2022-12-02,2022-12-03,2022-12-04,2022-12-05
3306,3810,"@johnredwood We can't frack, have new nuclear ...","@POTUS Moderating ??? Gas is going up again, o...",@OliviaG37717638 And the UK government priorit...,@RachelNotley High prices wouldn’t matter if y...,Sabah should install meter gauge to record oil...,@GBNEWS @LiamHalligan sunak and hunt trying to...,@XDoc5O @geemangeegee @GOPLeader Would you pre...,"@TamimBinHamad Dear Ursula Von Der Leyen, King...",Gordon sues Biden administration over oil and ...,@billywade5014 @EngelbertRober2 @GregAbbott_TX...,"Alex Christoforou Update: No reverse gear, EU ...",@POTUS The oil delivery guy just warned us tha...
3307,3813,@ThatgirlchrisC_ There are three mainparts in ...,@baseball31 Oh yeah you do and it is put you r...,@VincentH112 @Aidan_Regan @businessposthq Big ...,@mabuhaytrucking @mirandadevine what purpose f...,The Latest Oil and Gas News! https://t.co/s6f8...,🥚🍤🍒🍊🍯🍞🥐🥖🥞🍯🍞🥐🥖🍎🍐🍊🥫🍳🥚🍋🍌🍉🥦🍎🍍🍑🍇🍓🍅🥕🌽🍳🥚🍲Oil and gas ...,@DAMIADENUGA Say please I need money to start ...,"Crayfish, garri, cooking gas, vegetables and c...",@CafInsider @TheProcessLives the EU has paid r...,And how is @LancetCountdown and the many other...,"Gas prices are down, oil production is up, and...",@Carolyn33127771 @Swifter38191895 @GoodKharma1...
3308,3814,@HeatherRaymon20 There are three mainparts in ...,@PierrePoilievre Oil and gas or antithetical t...,Democrats are still in control of the Presiden...,@StephenKing Electric cars use more oil than g...,Europe and the US are deep into the hunt for e...,🥖🥞🍯🍞🥐🥖🍎🍐🍊🥫🍳🥚🍋🍌🍉🥦🍎🍍🍑🍇🍓🍅🥕🌽🍳🥚🍲🍐🍳🥚🍤Oil and gas bac...,@MKarnitschnig Pesky Americans kept them from ...,@xbasslichtie @YesEastAyrshire @RhodaMiller14 ...,Why are Gulf oil and gas producers so keen to ...,@RachelNotley OMG so true ❤ We should turn off...,@POTUS Now make an EV that charges as fast as ...,Oil and gas will still be the dominant source ...
3309,3815,Weakness in WCS price and Canadian oil stocks ...,@AniB64 @KyivIndependent That's the milion $ q...,@BMoens It is not the US's fault that Europe w...,@UGM043 @RBROYB139 @II_Ogirima @IjeleWills @St...,@castlvillageman Money!\nScotland's budget is ...,@GavinNewsom California has the highest per ga...,Five major oil refiners refuse to show up at s...,@TamimBinHamad @TebbouneAmadjid Dear Ursula Vo...,"If only he'd said ""this afternoon I met fossil...",@blue_highland Nothing to do with religion we ...,@POTUS ...with electricity largely created by ...,"@AP CHINA RUSSIA BIGGEST COAL, GAS, OIL, FOSSI..."
3310,3817,@AnneCroucher @CaryKelly11 There are three mai...,Reducing emissions from Canada's #oilandgas in...,@kaczyzm_2 @AMG26 @UserHomey @thesiriusreport ...,@groth1945 Not only that but where will the el...,"https://t.co/4woRdsEWrr ""The desire of the wes...",blah blah blah\nhttps://t.co/DyJuuuoVti @Eveni...,Mexico's Pemex increases production and import...,Russian Upstream Oil And Gas Investment Set To...,Mosman Oil &amp; Gas - Director Holdings #MSMN...,"@BladeoftheS Tories only worship themselves, m...",@Ronifae @sullim01 @AIWashburn @MichaelJStern1...,"Libya Oil, Gas and Renewable Energies Expo 202..."


In [16]:
#twitter_df_1d = np.array(twitter_df.reshape(43043, 1))
#twitter_df_1d = twitter_df.values.reshape(43043, 1)
switchbag = twitter_df.values.reshape(43043, 1)
twitter_df = pd.DataFrame(switchbag)

In [17]:
twitter_df.head()

,0
0,0
1,@BellaWallerstei They have been paying oil and...
2,@TrainerGonzo @Belinda71555585 It's that easy ...
3,@HouseGOP They tried to stick the gas price hi...
4,@GilHough @KamalaHarris https://t.co/LBgWDBW9xs


In [18]:
# Adding column name to tweets

twitter_df = twitter_df.rename(columns = {'0':'Tweets'})
twitter_df.head()

,0
0,0
1,@BellaWallerstei They have been paying oil and...
2,@TrainerGonzo @Belinda71555585 It's that easy ...
3,@HouseGOP They tried to stick the gas price hi...
4,@GilHough @KamalaHarris https://t.co/LBgWDBW9xs


In [22]:
twitter_df = pd.DataFrame(columns=['polarity_score', 'subjectivity_score'])

# Adding columns 
polarity_scores = []
subjectivity_scores = []
tweets = []

for tweet in tweets:
    doc = nlp(tweet)
    for token in doc:
        print

with open('tweets_clean.csv', 'r') as f:
    for line in tqdm(f, total=10000):
        tweet_dict, tweets, tweets_doc = 
        tweet_dict = json.loads(line)
        tweets = tweet_dict['text']
        tweets_doc = nlp(tweets)
        # 4. polarity score for each tweet (emotions expressed)
        polarity_scores.append(tweets_doc._.blob.polarity)
        # 5. subjectivity score for each tweet (personal feelings, views, beliefs expressed in the tweet)
        subjectivity_scores.append(tweets_doc._.blob.subjectivity)
        tweets.append(tweets)

twitter_df['polarity_score'] = polarity_scores
twitter_df['subjectivity_score'] = subjectivity_scores
twitter_df['tweets'] = tweets

  0%|          | 0/10000 [00:00<?, ?it/s]


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
twitter_df.head()

In [ ]:
# Drop NAs
twitter_df.dropna()

## Using spacy's English language model, individual tweets are tokenized and the following are collected:
1. total number of words
2. total number of punctuation marks
3. total number of words that are not stop words
4. polarity score (emotions expressed in the tweet)
5. subjectivity score (personal feelings, views, or beliefs expressed in the tweet)

In [ ]:
# 1. total number of words
def count_words(text):
    words = text.split()
    return len(words)

twitter_df['word_count'] = twitter_df["review"].apply(count_words)
twitter_df.head()

In [ ]:
# 2. total number of punctuation marks for each tweet
def count_punct_marks(text):
    char_count = 0
    for cha in text:
        if cha in PUNCTUATION:
            char_count+=1
    return char_count

twitter_df['punct_count'] = twitter_df["review"].apply(count_punct_marks)
twitter_df.head()

In [ ]:
# 3. total number of words that are not stop words for each tweet
def count_non_stop_words(text):
    non_stop_words = pd.Series(text.split())
    non_stop_words.str.strip(PUNCTUATION)

    drop_list = []
    for idx, word in enumerate(non_stop_words):
        if word in STOP_WORDS:
            drop_list.append(idx)
    non_stop_words.drop(index=drop_list, inplace=True)
    return len(non_stop_words)


twitter_df['non_stop_words'] = twitter_df["review"].apply(count_non_stop_words)
twitter_df.head()

## Description of Preliminary Data Preprocessing

## Data Examining & Cleaning 

# Sentiment Analysis Natural Language Processing Machine Learning Model

## Description of Preliminary Feature Engineering and Preliminary Feature Selection


# Explanation of Model Choice, including Limitations and Benefits 


In [ ]:
# X = 
# Y = 